N.B. This is a notebook I created for my talk at [Hopperx1 London 2019](https://londontechweek.com/event/hopperx1-london-people-and-skills) organised by fabulous people at [AnitaB.org](https://anitab.org/).

It is best served as an interactive HTML slide deck using [nbconvert](https://nbconvert.readthedocs.io/en/latest/install.html).

Here's how to do it. From your terminal, run:

```
$ jupyter nbconvert presentation/hopperx1.ipynb --to slides --post serve
```

It should open up a browser automatically. Modify the path to this notebook depending on where you're lauching it from.

In [1]:
###########
# Imports #
###########

#####################
# Utility functions #
#####################

<div class='title'>
    <h1 id='main-title'>Seeing is believing</h1>

    <h3 id='subtitle'>Neural nets, what do they "see"?</h3>

    <hr>

    <p>by Misa Ogura</p>
</div>

<scripts>
    <link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.8.2/css/all.css" integrity="sha384-oS3vJWv+0UjzBfQzYUhtDYW+Pj2yciDJxpsK1OYPAYjqT085Qq/1cq5FLXAZQ7Ay" crossorigin="anonymous">
</scripts>

<h1>Hello, I'm Misa 👋</h1>

<hr>

<div class='profile-container'>
    <img id='profile-image' src='images/misa_ogura_profile.jpg' width='200' height='200'>

    <ul id='profile-list'>
        <li>Originally from Tokyo, now based in London</li>
        <li>Cancer Cell Biologist, turned Software Engineer</li>
        <li>Currently at BBC R&D</li>
        <li>Co-founder of <a href='https://womendrivendev.org' target='_blank'>Women Driven Development</a></li>
        <li>Women in Data Science London Ambassador</li>
        <li>Occasionally writes</li>
    </ul>
</div>

## Convolutional Neural Networks (CNNs)

---

## Visualisation Techniques

---

## Demo

---